In [130]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta

In [131]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\output data'
namedataASCD_M = r'dataASCD_M.xlsx'
rutedataASCD_M = os.path.join(folder,namedataASCD_M)

In [132]:
dataASCD_M = pd.read_excel(rutedataASCD_M,header=0)

In [133]:
filterMT = dataASCD_M.loc[:,'Maintenance type'] != 'COND'
dataASCD_1TM = dataASCD_M.loc[filterMT,:].reset_index(drop=True)

In [135]:
dataASCD_1TM.loc[:,'Strategy'] = dataASCD_1TM.loc[:,'Frequency'].astype(int).astype(str) + dataASCD_1TM.loc[:,'Unidad Periodo']

dataASCD_1TM.loc[:,'Man hour'] = dataASCD_1TM.loc[:,'Time']*dataASCD_1TM.loc[:,'Amount Craft']

In [136]:
# Adding the pivot column to project
dataASCD_1TM.loc[:,'pivot'] = dataASCD_1TM.loc[:,'Asset tag'] + '-' + dataASCD_1TM.loc[:,'Strategy'] 
#dataASCD_1TP.loc[:,'pivot'] = dataASCD_1TP.loc[:,'Asset tag'] + '-' + dataASCD_1TP.loc[:,'Strategy']

In [137]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo','Strategy', 'Date task done']

In [138]:
projectionM = dataASCD_1TM.loc[:,['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']].groupby(['Asset tag','Strategy','Frequency', 'Unidad Periodo','Last date task done','Hour day']).size().reset_index()
projectionM.drop(columns = {0},inplace =True)
projectionM.loc[:,'Date plan'] = projectionM.loc[:,'Last date task done']
[row,column] = projectionM.shape

In [139]:
#A = pd.DataFrame(projectionM.loc[:,'Strategy'].unique(),columns=['Strategy'])

In [140]:
#B = pd.DataFrame(pd.date_range(start= date(2021,4,1),end=date(2021,4,9),freq='1d'),columns = ['date'])

In [141]:
#C = pd.merge(A,B,how='left',left_on='Strategy',right_on='date')

In [142]:
%%time
planDatesM = pd.DataFrame(columns = projectionM.columns)
projectionflag = projectionM.copy()
for m in range(row):
    while projectionflag.loc[m,'Date plan'] < date(2023,1,1):
        hour2days = round(projectionflag.loc[m,'Frequency']/projectionflag.loc[m,'Hour day'],0)
        if projectionflag.loc[m,'Unidad Periodo'] == 'H':
            delta=timedelta(days = hour2days)
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'S':
            delta = timedelta(weeks = projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'M':
            delta = timedelta(weeks = 4*projectionflag.loc[m,'Frequency'])
        elif  projectionflag.loc[m,'Unidad Periodo'] == 'A':
            delta = timedelta(weeks = 52*projectionflag.loc[m,'Frequency'])
        projectionflag.loc[m,'Date plan'] = projectionflag.loc[m,'Date plan'] + delta
        if projectionflag.loc[m,'Date plan'] < date(2023,1,1):
            planDatesM = planDatesM.append(projectionflag.loc[m,:],ignore_index=True)
        else:
            pass
planDatesM.loc[:,'pivot'] = planDatesM.loc[:,'Asset tag'] + '-' + planDatesM.loc[:,'Strategy']

Wall time: 52.9 s


In [143]:
planDatesM.shape

(13244, 8)

In [144]:
%%time
dataASCD_1TMPlan = pd.merge(dataASCD_1TM,planDatesM.loc[:,['pivot','Date plan']],how='left',on='pivot')
dataASCD_1TMPlan.to_excel('dataASCD_1TMPlan.xlsx', encoding='utf-8-sig',index=False)

Wall time: 40.6 s


In [145]:
dataASCD_1TMPlan.shape

(63116, 30)